In [1]:
import csv
import re, collections, math, datetime
import json
import collections

import sklearn
import sklearn.svm as svm
from sklearn import preprocessing
from sklearn.metrics import *

import numpy as np


def getDomainFeature(domain: str):
    feature = [domain.count('.'), len(domain), sum(c.isdigit() for c in domain), domain.count('-')]

    count_c = 0
    special_characters = (':', ';', '#', '!', '%', '~', '+', '_', '?', '=', '&', '[', ']')
    for c in domain:
        if c in special_characters:
            count_c = count_c + 1
    feature.append(count_c)

    s = len(domain)
    dd = collections.defaultdict(int)
    for c in domain:
        dd[c] += 1

    # 字符随机性
    # H(d) = － ∑lg( P( Xi ) ) * P( Xi )
    feature.append(sum(map(lambda value: -(value / s) * math.log2(value / s), dd.values())))

    # 元音字母比例
    feature.append(sum(map(lambda x: dd[x], ('a', 'e', 'i', 'o', 'u'))) / s)

    # 唯一字符数
    feature.append(len(dd))

    return feature


def getIPFeature(ip: str):
    if ':' in ip:
        x = ip.split(':')
        feature = x[0].split('.') + [x[1]]
    else:
        feature = ip.split('.') + [80]
    feature = list(map(lambda x: int(x), feature))
    return feature


if __name__ == "__main__":
    ip_pool = []
    ip_label = []

    domain_pool = []
    domain_label = []
    url_pool = []
    label_pool = []

    n = 0
    count = 0
    path1 = r"中国大学生服务外包创新创业大赛/网址自动分类识别/train1.csv"
    for _ in csv.reader(open(path1)):
        try:
            if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
                ip_pool.append(getIPFeature(_[0]))
                ip_label.append(int(_[1] == '0'))
            else:
                if count > 30000:
                    break
                domain_label.append(int(_[1] == '0'))
                domain_pool.append(getDomainFeature(_[0]))
                url_pool.append(_[0])
                if _[1] == '0':
                    count+=1
        except:
            print(_)

        n += 1
        if n % 100000 == 0:
            # break
            print(f"已读取分析{n}个数据")
            
            
    path2 = "data1add.csv"
    for _ in csv.reader(open(path2)):
        try:
            if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
                ip_pool.append(getIPFeature(_[0]))
                ip_label.append(int(_[1] == '0'))
            else:
                domain_label.append(int(_[1] == '0'))
                domain_pool.append(getDomainFeature(_[0]))
                url_pool.append(_[0])
        except:
            print(_)

        n += 1
        if n % 100000 == 0:
            # break
            print(f"已读取分析{n}个数据")
            
    
    print("读取数据完成")


读取数据完成


In [2]:

# ip_pool = []
# ip_label = []

# domain_pool = []
# domain_label = []
# url_pool = []

# n = 0
# path1 = r"中国大学生服务外包创新创业大赛/网址自动分类识别/train1.csv"
# for _ in csv.reader(open(path1)):
#     try:
#         if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
#             ip_pool.append(getIPFeature(_[0]))
#             ip_label.append(int(_[1] == '0'))
#         else:
#             domain_label.append(int(_[1] == '0'))
#             domain_pool.append(getDomainFeature(_[0]))
#             url_pool.append(_[0])
#     except:
#         print(_)

#     n += 1
#     if n % 100000 == 0:
#         # break
#         print(f"已读取分析{n}个数据")


# path2 = "data1add.csv"
# for _ in csv.reader(open(path2)):
#     try:
#         if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
#             ip_pool.append(getIPFeature(_[0]))
#             ip_label.append(int(_[1] == '0'))
#         else:
#             domain_label.append(int(_[1] == '0'))
#             domain_pool.append(getDomainFeature(_[0]))
#             url_pool.append(_[0])
#     except:
#         print(_)

#     n += 1
#     if n % 100000 == 0:
#         # break
#         print(f"已读取分析{n}个数据")


# print("读取数据完成")


In [3]:
# import numpy as np

# np.savetxt("domain_pool_large.txt",np.array(domain_pool))
# np.savetxt("domain_label_large.txt",np.array(domain_label))


In [4]:
with open('url_pool.txt', 'w+', encoding='utf-8') as f:
    for data in url_pool:
        # 添加‘\n’用于换行
        f.write(data+'\n')

In [5]:
print(len(url_pool))
print(len(domain_pool))

50105
50105


In [6]:
# domain_label = np.loadtxt("domain_label.txt")
# domain_pool = np.loadtxt("domain_pool.txt")
# print(domain_pool[0])
# print(domain_label2==domain_label)

In [7]:
def get_res_(values: list):
    value_cnt = {}  # 将结果用一个字典存储
    # 统计结果
    for value in values:
        # get(value, num)函数的作用是获取字典中value对应的键值, num=0指示初始值大小。
        value_cnt[value] = value_cnt.get(value, 0) + 1

    # 打印输出结果
    print(value_cnt)
    print([key for key in value_cnt.keys()])
    print([value for value in value_cnt.values()])


In [8]:
print(len(domain_pool))
# print(len([i for i in domain_label=='0'))
get_res_(domain_label)
get_res_(ip_label)

50105
{0: 20104, 1: 30001}
[0, 1]
[20104, 30001]
{0: 539, 1: 675}
[0, 1]
[539, 675]


In [9]:
# import torch
# import torch.nn.functional as F # 激励函数都在这
# # import tqdm
# import sklearn

# randomSeed = 1
# domain_size = len(domain_pool)
# domain_pool = preprocessing.scale(domain_pool)
# domain_pool, domain_label = sklearn.utils.shuffle(domain_pool, domain_label, random_state=randomSeed)

# class Net(torch.nn.Module): # 继承 torch 的 Module
#     def __init__(self, n_feature, n_hidden, n_output):
#         super(Net, self).__init__() # 继承 __init__ 功能
#         self.hidden = torch.nn.Linear(n_feature, n_hidden) # 隐藏层线性输出
#         self.hidden1 = torch.nn.Linear(n_hidden, n_hidden*3) # 隐藏层线性输出
#         self.hidden2 = torch.nn.Linear(n_hidden*3, n_hidden) # 隐藏层线性输出
#         self.hidden3 = torch.nn.Linear(n_hidden, n_hidden) # 隐藏层线性输出
#         self.out = torch.nn.Linear(n_hidden, n_output) # 输出层线性输出
#     def forward(self, x):
#         # 正向传播输入值, 神经网络分析出输出值
#         x = F.relu(self.hidden(x)) # 激励函数(隐藏层的线性值)
#         x = F.relu(self.hidden1(x))
#         x = F.relu(self.hidden2(x))
#         x = F.relu(self.hidden3(x))
#         x = self.out(x) # 输出值, 但是这个不是预测值, 预测值还需要再另外计算
#         return x
# net = Net(n_feature=8, n_hidden=10, n_output=2) # 几个类别就几个 output
# print(net)

# x=torch.Tensor(domain_pool[:int(domain_size * 0.8)]).type(torch.FloatTensor) 
# y=torch.Tensor(domain_label[:int(domain_size * 0.8)]).type(torch.LongTensor)
# x_test=torch.Tensor(domain_pool[int(domain_size * 0.8):]).type(torch.FloatTensor)
# y_test=torch.Tensor(domain_label[int(domain_size * 0.8):]).type(torch.LongTensor)

# optimizer = torch.optim.SGD(net.parameters(), lr=0.02) #学习率等
# loss_func = torch.nn.CrossEntropyLoss() #交叉熵损失函数
# # loss_func = focal_loss(alpha=0.25, gamma=2, num_classes=2)
# for t in range(100):
#     out = net(x) 
#     loss = loss_func(out, y) # (1. nn output, 2. target)
#     optimizer.zero_grad() # 清空梯度
#     loss.backward() # 反向
#     optimizer.step() # 应用梯度
#     print(t,end=" ")
    
#     if t%1==0:
#         prediction = torch.max(out, 1)[1]
#         # print(prediction)
        
#         pred_y = prediction.data.numpy()
#         # print(pred_y)
#         label_y = y.data.numpy()

#         now = datetime.datetime.now()
#         timestamp = datetime.datetime.timestamp(now)
#         TF = {"timestamp":timestamp,
#               "randomSeed":randomSeed,
#               "训练集规模":int(domain_size * 0.8),
#               "测试集规模":domain_size - int(domain_size * 0.8),
#               "标准化均值":tuple(domain_pool.mean(axis=0)),
#               "标准化方差":tuple(domain_pool.std(axis=0)),
#               "准确率": accuracy_score(label_y, pred_y),
#               "f1": f1_score(label_y, pred_y),
#               "f1_macro": f1_score(label_y, pred_y,average="macro"),
#               "recall_macro": recall_score(label_y, pred_y,average="macro"),
#               "roc_macro": roc_auc_score(label_y, pred_y,average="macro"),
#               "precision_macro": precision_score(label_y, pred_y,average="macro"),
#               "recall": recall_score(label_y, pred_y),
#               "roc": roc_auc_score(label_y, pred_y),
#               "precision": precision_score(label_y, pred_y),
#               "模型名称": "NN"
#               }

#         with open("log_"+TF["模型名称"]+".json","w") as f:
#             b = json.dumps(TF)
#             f.write(b)

#         print(f"测试集\033[0;34m准确率\033[0m为\033[0;31m{TF['准确率'] * 100:.2f}%")
#         print(f"\033[0m测试集\033[0;34mf1值\033[0m为\033[0;31m{TF['f1'] * 100:.2f}%")
#         print(f"\033[0m测试集\033[0;34mrecall值\033[0m为\033[0;31m{TF['recall'] * 100:.2f}%")
#         print(f"\033[0m测试集\033[0;34mroc值\033[0m为\033[0;31m{TF['roc'] * 100:.2f}%")
#         print(f"\033[0m测试集\033[0;34mprecision值\033[0m为\033[0;31m{TF['precision'] * 100:.2f}%\033[0m")
#         print(f"\033[0m测试集\033[0;34mf1_macro值\033[0m为\033[0;31m{TF['f1_macro'] * 100:.2f}%")
#         print(f"\033[0m测试集\033[0;34mrecall_macro值\033[0m为\033[0;31m{TF['recall_macro'] * 100:.2f}%")
#         print(f"\033[0m测试集\033[0;34mroc_macro值\033[0m为\033[0;31m{TF['roc_macro'] * 100:.2f}%")
#         print(f"\033[0m测试集\033[0;34mprecision_macro值\033[0m为\033[0;31m{TF['precision_macro'] * 100:.2f}%\033[0m")

# # prediction = torch.max(out, 1)[1]
# # #print(prediction)
# # pred_y = prediction.data.numpy()
# # target_y = y.data.numpy()
# # # plt.scatter(x.data.numpy()[:, 0], x.data.numpy()[:, 1], c=pred_y, s=100, lw=0, cmap='rainbow')
# # accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)

# # print( '训练集准确度：',accuracy)
# # # plt.show()

# # out=net(x_test)
# # prediction = torch.max(out, 1)[1]
# # pred_y = prediction.data.numpy()
# # ta_y = y_test.data.numpy()
# # # plt.scatter(x_test.data.numpy()[:, 0], x_test.data.numpy()[:, 1], c=pred_y, s=100, lw=0, cmap='rain
# # accuracy = float((pred_y == ta_y).astype(int).sum()) / float(ta_y.size)
# # print( '测试集准确度：',accuracy)
# # plt.show()

In [10]:
domain_svm_model = sklearn.linear_model.LogisticRegression(verbose=True)

randomSeed = 1
domain_size = len(domain_pool)
domain_pool = preprocessing.scale(domain_pool)
domain_pool, domain_label = sklearn.utils.shuffle(domain_pool, domain_label, random_state=randomSeed)

# 训练模型,x为训练集,y为标签
domain_svm_model.fit(domain_pool[:int(domain_size * 0.8)], domain_label[:int(domain_size * 0.8)])
print("训练模型完成")

pred_y = domain_svm_model.predict(domain_pool[int(domain_size * 0.8):])
label_y = domain_label[int(domain_size * 0.8):]

now = datetime.datetime.now()
timestamp = datetime.datetime.timestamp(now)

TF = {"timestamp":timestamp,
      "randomSeed":randomSeed,
      "训练集规模":int(domain_size * 0.8),
      "测试集规模":domain_size - int(domain_size * 0.8),
      "标准化均值":tuple(domain_pool.mean(axis=0)),
      "标准化方差":tuple(domain_pool.std(axis=0)),
      "准确率": accuracy_score(label_y, pred_y),
      "f1": f1_score(label_y, pred_y),
      "recall": recall_score(label_y, pred_y),
      "roc": roc_auc_score(label_y, pred_y),
      "precision": precision_score(label_y, pred_y),
      "模型名称": "logistic regression"
      }

with open("log_"+TF["模型名称"]+".json","w") as f:
    b = json.dumps(TF)
    f.write(b)

print(f"测试集\033[0;34m准确率\033[0m为\033[0;31m{TF['准确率'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mf1值\033[0m为\033[0;31m{TF['f1'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mrecall值\033[0m为\033[0;31m{TF['recall'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mroc值\033[0m为\033[0;31m{TF['roc'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mprecision值\033[0m为\033[0;31m{TF['precision'] * 100:.2f}%")


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.77841D+04    |proj g|=  1.65879D+04


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     35     39      1     0     0   2.737D-02   7.046D+03
  F =   7045.7546134826689     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
训练模型完成


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.1s finished


测试集准确率为93.28%
测试集f1值为94.26%
测试集recall值为91.56%
测试集roc值为93.72%
测试集precision值为97.12%


In [11]:
import joblib

joblib.dump(domain_svm_model, 'logistic_regression_model.pkl')

['logistic_regression_model.pkl']

In [12]:
get_res_(pred_y)
get_res_(label_y)
print(pred_y[:1100])
get_res_(pred_y[:1100])

{0: 4336, 1: 5685}
[0, 1]
[4336, 5685]
{0: 3991, 1: 6030}
[0, 1]
[3991, 6030]
[0 0 0 ... 0 1 1]
{0: 494, 1: 606}
[0, 1]
[494, 606]


## 对全部数据集train1和train1补充进行过滤

In [23]:
import csv
import re, collections, math, datetime
import json
import collections

import sklearn
import sklearn.svm as svm
from sklearn import preprocessing
from sklearn.metrics import *

import numpy as np


def getDomainFeature(domain: str):
    feature = [domain.count('.'), len(domain), sum(c.isdigit() for c in domain), domain.count('-')]

    count_c = 0
    special_characters = (':', ';', '#', '!', '%', '~', '+', '_', '?', '=', '&', '[', ']')
    for c in domain:
        if c in special_characters:
            count_c = count_c + 1
    feature.append(count_c)

    s = len(domain)
    dd = collections.defaultdict(int)
    for c in domain:
        dd[c] += 1

    # 字符随机性
    # H(d) = － ∑lg( P( Xi ) ) * P( Xi )
    feature.append(sum(map(lambda value: -(value / s) * math.log2(value / s), dd.values())))

    # 元音字母比例
    feature.append(sum(map(lambda x: dd[x], ('a', 'e', 'i', 'o', 'u'))) / s)

    # 唯一字符数
    feature.append(len(dd))

    return feature


def getIPFeature(ip: str):
    if ':' in ip:
        x = ip.split(':')
        feature = x[0].split('.') + [x[1]]
    else:
        feature = ip.split('.') + [80]
    feature = list(map(lambda x: int(x), feature))
    return feature


if __name__ == "__main__":
    ip_pool = []
    ip_label = []

    domain_pool = []
    domain_label = []
    url_pool = []
    label_pool = []

    n = 0
    count = 0
    path1 = r"中国大学生服务外包创新创业大赛/网址自动分类识别/train1.csv"
    for _ in csv.reader(open(path1)):
        try:
            if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
                ip_pool.append(getIPFeature(_[0]))
                ip_label.append(int(_[1] == '0'))
            else:
                domain_label.append(int(_[1] == '0'))
                domain_pool.append(getDomainFeature(_[0]))
                url_pool.append(_[0])
                label_pool.append(_[1])
                if _[1] == '0':
                    count+=1
        except:
            print(_)

        n += 1
        if n % 100000 == 0:
            # break
            print(f"已读取分析{n}个数据")
            
            
    path2 = "data1add.csv"
    for _ in csv.reader(open(path2)):
        try:
            if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
                ip_pool.append(getIPFeature(_[0]))
                ip_label.append(int(_[1] == '0'))
            else:
                domain_label.append(int(_[1] == '0'))
                domain_pool.append(getDomainFeature(_[0]))
                url_pool.append(_[0])
                label_pool.append(_[1])
        except:
            print(_)

        n += 1
        if n % 100000 == 0:
            # break
            print(f"已读取分析{n}个数据")
    
    print("读取数据完成")


已读取分析100000个数据
已读取分析200000个数据
已读取分析300000个数据
已读取分析400000个数据
已读取分析500000个数据
已读取分析600000个数据
已读取分析700000个数据
已读取分析800000个数据
已读取分析900000个数据
已读取分析1000000个数据
已读取分析1100000个数据
已读取分析1200000个数据
已读取分析1300000个数据
已读取分析1400000个数据
已读取分析1500000个数据
已读取分析1600000个数据
已读取分析1700000个数据
已读取分析1800000个数据
已读取分析1900000个数据
已读取分析2000000个数据
已读取分析2100000个数据
已读取分析2200000个数据
已读取分析2300000个数据
已读取分析2400000个数据
已读取分析2500000个数据
已读取分析2600000个数据
已读取分析2700000个数据
已读取分析2800000个数据
已读取分析2900000个数据
已读取分析3000000个数据
已读取分析3100000个数据
已读取分析3200000个数据
已读取分析3300000个数据
已读取分析3400000个数据
已读取分析3500000个数据
已读取分析3600000个数据
已读取分析3700000个数据
已读取分析3800000个数据
已读取分析3900000个数据
已读取分析4000000个数据
已读取分析4100000个数据
已读取分析4200000个数据
已读取分析4300000个数据
已读取分析4400000个数据
已读取分析4500000个数据
已读取分析4600000个数据
已读取分析4700000个数据
已读取分析4800000个数据
已读取分析4900000个数据
已读取分析5000000个数据
已读取分析5100000个数据
已读取分析5200000个数据
已读取分析5300000个数据
已读取分析5400000个数据
已读取分析5500000个数据
已读取分析5600000个数据
已读取分析5700000个数据
已读取分析5800000个数据
已读取分析5900000个数据
已读取分析6000000个数据
已读取分析6100000个数据
已读取分析6200000个数据
已读取分析6300000个数据
已

In [14]:
print(len(domain_pool))
print(len(domain_label))
print(len(url_pool))
print(len(label_pool))

8345817
8345817
8345817


In [10]:
# import numpy as np

# domain_label_a =np.loadtxt("domain_label_large.txt")
# domain_pool_a = np.loadtxt("domain_pool_large.txt")
# url_pool = []
# with open('url_pool.txt', 'r', encoding='utf-8') as f:
#     for data in f:
#         url_pool.append(data[:-1])
        
# l_pool = []
# with open('label_all.txt', 'r', encoding='utf-8') as f:
#     for data in f:
#         l_pool.append(data[:-1])
# print(len(l_pool))

16691634


In [11]:
# l_pool = []
# with open('label-all.txt', 'r', encoding='utf-8') as f:
#     for data in f:
#         l_pool.append(data[:-1])

In [12]:
# print(len(l_pool))
# print(len(url_pool))

8345817
25037451


In [15]:
domain_label = np.asarray(domain_label,dtype=np.int32)

In [16]:
domain_pool = preprocessing.scale(domain_pool)

pred_y = domain_svm_model.predict(domain_pool)
pred_y_p = domain_svm_model.predict_proba(domain_pool)

label_y = domain_label

now = datetime.datetime.now()
timestamp = datetime.datetime.timestamp(now)
TF = {"timestamp":timestamp,
      "randomSeed":randomSeed,
      "训练集规模":int(domain_size * 0.8),
      "测试集规模":domain_size - int(domain_size * 0.8),
      "标准化均值":tuple(domain_pool.mean(axis=0)),
      "标准化方差":tuple(domain_pool.std(axis=0)),
      "准确率": accuracy_score(label_y, pred_y),
      "f1": f1_score(label_y, pred_y),
      "recall": recall_score(label_y, pred_y),
      # "roc": roc_auc_score(label_y, pred_y),
      "precision": precision_score(label_y, pred_y),
      "模型名称":"logistic regression all-130w-new"
      
      }

with open("log_"+TF["模型名称"]+".json","w") as f:
    b = json.dumps(TF)
    f.write(b)

print(f"测试集\033[0;34m准确率\033[0m为\033[0;31m{TF['准确率'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mf1值\033[0m为\033[0;31m{TF['f1'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mrecall值\033[0m为\033[0;31m{TF['recall'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mroc值\033[0m为\033[0;31m{TF['roc'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mprecision值\033[0m为\033[0;31m{TF['precision'] * 100:.2f}%")

# TF = {"timestamp":timestamp,
#       "randomSeed":randomSeed,
#       # "训练集规模":int(domain_size * 0.8),
#       # "测试集规模":domain_size - int(domain_size * 0.8),
#       "标准化均值":tuple(domain_pool.mean(axis=0)),
#       "标准化方差":tuple(domain_pool.std(axis=0)),
#       "准确率": accuracy_score(label_y, pred_y),
#       "f1": f1_score(label_y, pred_y),
#       "f1_macro": f1_score(label_y, pred_y,average="macro"),
#       "recall_macro": recall_score(label_y, pred_y,average="macro"),
#       "roc_macro": roc_auc_score(label_y, pred_y,average="macro"),
#       "precision_macro": precision_score(label_y, pred_y,average="macro"),
#       "recall": recall_score(label_y, pred_y),
#       "roc": roc_auc_score(label_y, pred_y),
#       "precision": precision_score(label_y, pred_y),
#       "模型名称": "logistic regression all"
#       }

# with open("log_"+TF["模型名称"]+".json","w") as f:
#     b = json.dumps(TF)
#     f.write(b)

# print(f"测试集\033[0;34m准确率\033[0m为\033[0;31m{TF['准确率'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mf1值\033[0m为\033[0;31m{TF['f1'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mrecall值\033[0m为\033[0;31m{TF['recall'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mroc值\033[0m为\033[0;31m{TF['roc'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mprecision值\033[0m为\033[0;31m{TF['precision'] * 100:.2f}%\033[0m")
# print(f"\033[0m测试集\033[0;34mf1_macro值\033[0m为\033[0;31m{TF['f1_macro'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mrecall_macro值\033[0m为\033[0;31m{TF['recall_macro'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mroc_macro值\033[0m为\033[0;31m{TF['roc_macro'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mprecision_macro值\033[0m为\033[0;31m{TF['precision_macro'] * 100:.2f}%\033[0m")


测试集准确率为84.51%
测试集f1值为91.58%
测试集recall值为84.47%
测试集precision值为100.00%


In [17]:
print(len(label_y))
print(len(pred_y))
get_res_(pred_y)
get_res_(label_y)
print(len(pred_y_p))
print(pred_y_p)
# print(np.where(pred_y_p))

8345817
8345817
{0: 1312818, 1: 7032999}
[0, 1]
[1312818, 7032999]
{0: 20104, 1: 8325713}
[0, 1]
[20104, 8325713]
8345817
[[9.99972632e-01 2.73675876e-05]
 [9.99892211e-01 1.07788643e-04]
 [9.99895329e-01 1.04671438e-04]
 ...
 [9.95886384e-01 4.11361567e-03]
 [9.99974933e-01 2.50666577e-05]
 [9.99866135e-01 1.33864500e-04]]


In [24]:
suspect_pool=[]
s_label=[]
print(len(domain_pool))
print(len(domain_label))
print(len(url_pool))
for n,i in enumerate(pred_y):
    if i==0:
        suspect_pool.append(url_pool[n])
        s_label.append(label_pool[n])

8345817
8345817
8345817


In [25]:
print(len(suspect_pool))
print(len(s_label))

1312818
1312818


In [26]:
with open('suspect_pool_new.txt', 'w', encoding='utf-8') as f:
    for i in range(len(suspect_pool)):
        # 添加‘\n’用于换行
        f.write(suspect_pool[i]+","+str(s_label[i])+'\n')

In [20]:
# domain_label_a =np.loadtxt("domain_label_large.txt")
domain_pool_a = np.loadtxt("domain_pool_test.txt")
url_pool = []
with open('url_pool_test.txt', 'r', encoding='utf-8') as f:
    for data in f:
        url_pool.append(data[:-1])

domain_pool_a = preprocessing.scale(domain_pool_a)
pred_y = domain_svm_model.predict(domain_pool_a)

In [21]:
print(len(pred_y))
get_res_(pred_y)
print(len(url_pool))

1025767
{0: 161381, 1: 864386}
[0, 1]
[161381, 864386]
1025767


In [22]:
url_susp=[]
for i in range(len(pred_y)):
    if pred_y[i]==0:
        url_susp.append(url_pool[i])

In [23]:
print(len(url_susp))

161381


In [24]:
# with open('url_suspect_test.txt', 'w+', encoding='utf-8') as f:
#     f.write("\n".join(url_susp))

## 换用SVM，暂时废弃

In [25]:
# 建立模型

domain_svm_model = svm.SVC(C=1, kernel='rbf',verbose=True)

randomSeed = 1
domain_size = len(domain_pool)
domain_pool = preprocessing.scale(domain_pool)
domain_pool, domain_label = sklearn.utils.shuffle(domain_pool, domain_label, random_state=randomSeed)

# 训练模型,x为训练集,y为标签
domain_svm_model.fit(domain_pool[:int(domain_size * 0.8)], domain_label[:int(domain_size * 0.8)])
print("训练模型完成")

pred_y = domain_svm_model.predict(domain_pool[int(domain_size * 0.8):])
label_y = domain_label[int(domain_size * 0.8):]

now = datetime.datetime.now()
timestamp = datetime.datetime.timestamp(now)

TF = {"timestamp":timestamp,
      "randomSeed":randomSeed,
      "训练集规模":int(domain_size * 0.8),
      "测试集规模":domain_size - int(domain_size * 0.8),
      "标准化均值":tuple(domain_pool.mean(axis=0)),
      "标准化方差":tuple(domain_pool.std(axis=0)),
      "准确率": accuracy_score(label_y, pred_y),
      "f1": f1_score(label_y, pred_y),
      "f1_macro": f1_score(label_y, pred_y,average="macro"),
      "recall_macro": recall_score(label_y, pred_y,average="macro"),
      "roc_macro": roc_auc_score(label_y, pred_y,average="macro"),
      "precision_macro": precision_score(label_y, pred_y,average="macro"),
      "recall": recall_score(label_y, pred_y),
      "roc": roc_auc_score(label_y, pred_y),
      "precision": precision_score(label_y, pred_y),
      "模型名称": "SVM"
      }

# with open("log_"+TF["模型名称"]+".json","w") as f:
#     b = json.dumps(TF)
#     f.write(b)

print(f"测试集\033[0;34m准确率\033[0m为\033[0;31m{TF['准确率'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mf1值\033[0m为\033[0;31m{TF['f1'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mrecall值\033[0m为\033[0;31m{TF['recall'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mroc值\033[0m为\033[0;31m{TF['roc'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mprecision值\033[0m为\033[0;31m{TF['precision'] * 100:.2f}%\033[0m")
print(f"\033[0m测试集\033[0;34mf1_macro值\033[0m为\033[0;31m{TF['f1_macro'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mrecall_macro值\033[0m为\033[0;31m{TF['recall_macro'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mroc_macro值\033[0m为\033[0;31m{TF['roc_macro'] * 100:.2f}%")
print(f"\033[0m测试集\033[0;34mprecision_macro值\033[0m为\033[0;31m{TF['precision_macro'] * 100:.2f}%\033[0m")


[LibSVM]......
*....
*.
*
optimization finished, #iter = 10276
obj = -4831.333110, rho = -0.133689
nSV = 5157, nBSV = 4933
Total nSV = 5157
训练模型完成
测试集准确率为94.90%
测试集f1值为95.62%
测试集recall值为92.85%
测试集roc值为95.41%
测试集precision值为98.55%
测试集f1_macro值为94.76%
测试集recall_macro值为95.41%
测试集roc_macro值为95.41%
测试集precision_macro值为94.36%


In [26]:
# domain_label = np.loadtxt("domain_label.txt")
# domain_pool = np.loadtxt("domain_pool.txt")


# pred_y = domain_svm_model.predict(domain_pool[int(domain_size * 0.8):])
# label_y = domain_label[int(domain_size * 0.8):]

# now = datetime.datetime.now()
# timestamp = datetime.datetime.timestamp(now)

# TF = {"timestamp":timestamp,
#       "randomSeed":randomSeed,
#       "训练集规模":int(domain_size * 0.8),
#       "测试集规模":domain_size - int(domain_size * 0.8),
#       "标准化均值":tuple(domain_pool.mean(axis=0)),
#       "标准化方差":tuple(domain_pool.std(axis=0)),
#       "准确率": accuracy_score(label_y, pred_y),
#       "f1": f1_score(label_y, pred_y),
#       "f1_macro": f1_score(label_y, pred_y,average="macro"),
#       "recall_macro": recall_score(label_y, pred_y,average="macro"),
#       "roc_macro": roc_auc_score(label_y, pred_y,average="macro"),
#       "precision_macro": precision_score(label_y, pred_y,average="macro"),
#       "recall": recall_score(label_y, pred_y),
#       "roc": roc_auc_score(label_y, pred_y),
#       "precision": precision_score(label_y, pred_y),
#       "模型名称": "SVM-all"
#       }

# with open("log_"+TF["模型名称"]+".json","w") as f:
#     b = json.dumps(TF)
#     f.write(b)

# print(f"测试集\033[0;34m准确率\033[0m为\033[0;31m{TF['准确率'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mf1值\033[0m为\033[0;31m{TF['f1'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mrecall值\033[0m为\033[0;31m{TF['recall'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mroc值\033[0m为\033[0;31m{TF['roc'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mprecision值\033[0m为\033[0;31m{TF['precision'] * 100:.2f}%\033[0m")
# print(f"\033[0m测试集\033[0;34mf1_macro值\033[0m为\033[0;31m{TF['f1_macro'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mrecall_macro值\033[0m为\033[0;31m{TF['recall_macro'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mroc_macro值\033[0m为\033[0;31m{TF['roc_macro'] * 100:.2f}%")
# print(f"\033[0m测试集\033[0;34mprecision_macro值\033[0m为\033[0;31m{TF['precision_macro'] * 100:.2f}%\033[0m")


# 处理test（第一阶段）

In [27]:
import csv
import re, collections, math, datetime
import json
import collections

import sklearn
import sklearn.svm as svm
from sklearn import preprocessing
from sklearn.metrics import *

import numpy as np


def getDomainFeature(domain: str):
    feature = [domain.count('.'), len(domain), sum(c.isdigit() for c in domain), domain.count('-')]

    count_c = 0
    special_characters = (':', ';', '#', '!', '%', '~', '+', '_', '?', '=', '&', '[', ']')
    for c in domain:
        if c in special_characters:
            count_c = count_c + 1
    feature.append(count_c)

    s = len(domain)
    dd = collections.defaultdict(int)
    for c in domain:
        dd[c] += 1

    # 字符随机性
    # H(d) = － ∑lg( P( Xi ) ) * P( Xi )
    feature.append(sum(map(lambda value: -(value / s) * math.log2(value / s), dd.values())))

    # 元音字母比例
    feature.append(sum(map(lambda x: dd[x], ('a', 'e', 'i', 'o', 'u'))) / s)

    # 唯一字符数
    feature.append(len(dd))

    return feature


if __name__ == "__main__":
    ip_pool = []
    # ip_label = []

    domain_pool = []
    # domain_label = []
    url_pool = []

    n = 0
    count = 0
    path1 = r"中国大学生服务外包创新创业大赛/网址自动分类识别/test(unlabeled).csv"
    for _ in csv.reader(open(path1)):
        try:
            if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
                # print(1)
                ip_pool.append(_[0])
                # ip_label.append(int(_[1] == '0'))
            else:
                # if count > 30000:
                #     break
                # domain_label.append(int(_[1] == '0'))
                domain_pool.append(getDomainFeature(_[0]))
                url_pool.append(_[0])
                # if _[1] == '0':
                #     count+=1
        except:
            print(",1",_[0])

        n += 1
        if n % 100000 == 0:
            # break
            print(f"已读取分析{n}个数据")
            
            
    # path2 = "data1add.csv"
    # for _ in csv.reader(open(path2)):
    #     try:
    #         if re.match(r"^\d+\.\d+\.\d+\.\d+(:\d+)?$", _[0]):
    #             ip_pool.append(getIPFeature(_[0]))
    #             # ip_label.append(int(_[1] == '0'))
    #         else:
    #             # domain_label.append(int(_[1] == '0'))
    #             domain_pool.append(getDomainFeature(_[0]))
    #             url_pool.append(_[0])
    #     except:
    #         print("1",_)

        # n += 1
        # if n % 100000 == 0:
        #     # break
        #     print(f"已读取分析{n}个数据")
    
    print("读取数据完成")


已读取分析100000个数据
已读取分析200000个数据
已读取分析300000个数据
已读取分析400000个数据
已读取分析500000个数据
已读取分析600000个数据
已读取分析700000个数据
已读取分析800000个数据
已读取分析900000个数据
已读取分析1000000个数据
读取数据完成


In [28]:
# import numpy as np

# np.savetxt("domain_pool_test.txt",np.array(domain_pool))
# with open("url_pool_test.txt","w+",encoding="UTF-8") as f:
#     f.write("\n".join(url_pool))
# # np.savetxt("url_pool_test.txt",np.array(url_pool))
# with open("ip_pool_test.txt","w+",encoding="UTF-8") as f:
#     f.write("\n".join(ip_pool))
# # np.savetxt("ip_pool_test.txt",np.array(ip_pool))
